### Prepare inputs for discretization ###

Prepare input data for the domain area (except radiation). This step includes:<br>
1. project large-domain input rasters and vectors into one projection system.
2. project the domain gru shaefile for extraction.
3. extract/crop inputs for the domain area.
4. resample inputs according to a reference raster (if needed).

In [1]:
# import libraries
import os
import geospatial_functions.geospatial_analysis as ga
import geospatial_functions.utils as ut
import rasterio as rio
import rasterio.shutil
from rasterio.warp import Resampling

In [2]:
# common paths
control_file = 'control_active.txt'
root_path = ut.read_from_control(control_file, 'root_path')
source_path = ut.read_from_control(control_file, 'source_path')
domain_name = ut.read_from_control(control_file, 'domain_name')
domain_path = os.path.join(root_path, domain_name)

In [3]:
# projection system
proj4=ut.read_from_control(control_file, 'proj4') 
dst_crs = rio.crs.CRS.from_string(proj4)

In [4]:
# large-domain data
landcover_raster = ut.read_from_control(control_file, 'landcover_raster')
landcover_clip_raster = os.path.join(source_path, os.path.basename(landcover_raster).split('.tif')[0]+'_clip.tif')
landcover_prj_raster =  ut.read_from_control(control_file, 'landcover_prj_raster')
if landcover_prj_raster == 'default':
    landcover_prj_raster = os.path.join(source_path, os.path.basename(landcover_raster).split('.tif')[0]+'_prj.tif')

# domain data
domain_gru_prj_shp = ut.specify_file_path(control_file, 'domain_gru_prj_shp')
domain_extent = ut.specify_file_path(control_file, 'domain_extent')
domain_landcover_raster = ut.specify_file_path(control_file, 'domain_landcover_raster') 
domain_landcover_resample_raster = ut.specify_file_path(control_file, 'domain_landcover_resample_raster')
domain_landcover_class_raster = ut.specify_file_path(control_file, 'domain_landcover_class_raster')

#### 1. Clip extent landcover from large-domain input ####

In [30]:
# (1) reproject raster by creating a VRT file, which is merely a ASCII txt file, for easy handling --- 
inraster = landcover_raster
dst_crs_geo = rio.crs.CRS.from_string("EPSG:4326")
inraster_vrt = os.path.join(os.path.dirname(inraster), 
                            os.path.basename(inraster).split('.tif')[0]+'.vrt')
with rio.open(inraster) as src:
    with rio.vrt.WarpedVRT(src, crs=dst_crs_geo, resampling=rio.enums.Resampling.nearest) as vrt:
        rio.shutil.copy(vrt, inraster_vrt, driver='VRT')
    
# (2) Do the cut out based on user defined extent
[lat_max,lon_min,lat_min,lon_max] = domain_extent.split('/')
inraster_vrt_clip = os.path.join(os.path.dirname(inraster),
                                 os.path.basename(inraster).split('.')[0]+'_clip.vrt')

cmd = 'gdal_translate -of VRT -projwin %s %s %s %s %s %s'%\
(lon_min, lat_max, lon_max, lat_min,inraster_vrt,inraster_vrt_clip)
os.system(cmd)

# (3) convert vrt to raster
inraster_clip = landcover_clip_raster
cmd = 'gdal_translate %s %s'%(inraster_vrt_clip, inraster_clip)
os.system(cmd)

0

#### 2. Project extent landcover #### 

In [31]:
ga.reproject_raster(landcover_clip_raster, landcover_prj_raster, dst_crs, Resampling.nearest) 

#### 3. Extract domain landcover ####

In [32]:
ga.crop_raster(landcover_prj_raster, domain_gru_prj_shp, domain_landcover_raster) 

#### 4. Resample domain landcover ####
Resample is only applied to the domain raster that has a coarser resolution than the reference raster. The reference raster is specified by user (by default, it refers to the domain DEM).

In [33]:
refraster = ut.specify_refraster_path(control_file)
ga.resample_raster(domain_landcover_raster,refraster,domain_landcover_resample_raster) 

#### 5. Classify domain landcover ####

In [6]:
# Here two classes were made: crop vs. non-crop.
# Crop class includes: 0 Evergreen needleleaf forests, 1 Evergreen broadleaf forests, 
# 2 Deciduous needleleaf forests, 3 Deciduous broadleaf forests, 4 Mixed forests, 6 Closed shrublands, 7 Woody savannas.  
ga.classify_landcover(domain_landcover_resample_raster, domain_landcover_class_raster)